# Emoji Predictor

### Problem Description

For a given sentence predict a most suitable emoji that represents the sentiment present in that sentences. 

In [30]:
# Importing all required libraries.
import pandas as pd
import numpy as np
import emoji
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import *

In [31]:
# Reading dataset in csv form using pandas.
train = pd.read_csv('Datasets/Train_emoji.csv',header=None)
test = pd.read_csv('Datasets/Test_emoji.csv',header=None)

In [32]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [33]:
test.head()

,0,1
0,I want to eat\t,4
1,he did not answer\t,3
2,he got a raise\t,2
3,she got me a present\t,0
4,ha ha ha it was so funny\t,2


In [34]:
# Data cleaning.
XTrain = train[0]
XTest = test[0]
YTrain = to_categorical(train[1])
YTest = to_categorical(test[1])

print(XTrain.shape)
print(XTest.shape)
print(YTrain.shape)
print(YTest.shape)

(132,)
(56,)
(132, 5)
(56, 5)


In [35]:
# Creating dictionary to map emoji to numbers for classification.
emoji_dictionary = { "0" : "\u2764\uFE0F",
                     "1" : ":baseball:",
                     "2" : ":beaming_face_with_smiling_eyes:",
                     "3" : ":downcast_face_with_sweat:",
                     "4" : ":fork_and_knife:"
}

In [36]:
# Creating embedding_index using glove.6B.50d.txt for making embedding layer. We have used the concept of transfer learning.
f = open('Embedding/glove.6B.50d.txt',encoding='utf-8')
embedding_index = {}
cnt = 0
for line in f :
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float32')
    embedding_index[word] = coefs
f.close()

In [37]:
# Creating function for getting output for embedding layer.
emb_dim = embedding_index["eat"].shape[0]
def embedding_output(X):
    maxLen = 10
    embedding_out = np.zeros((X.shape[0],maxLen,emb_dim))
    for i in range(X.shape[0]):
        X[i] = X[i].split()
        for j in range(len(X[i])):
            try:
                embedding_out[i][j] = embedding_index[X[i][j].lower()]
            except:
                embedding_out[i][j] = np.zeros((50,))
    return embedding_out

In [38]:
# Output for embedding layer.
emb_XTrain = embedding_output(XTrain)
emb_XTest = embedding_output(XTest)

C:\Users\Preetish Vij\AppData\Local\Temp\ipykernel_3168\4094838632.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[i] = X[i].split()


In [39]:
print(emb_XTrain.shape)
print(emb_XTest.shape)

(132, 10, 50)
(56, 10, 50)


In [40]:
# Creating an LSTM model for predicting emoji. 
model = Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64,return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 10, 64)            29440     
                                                                 
 dropout_4 (Dropout)         (None, 10, 64)            0         
                                                                 
 lstm_5 (LSTM)               (None, 64)                33024     
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 5)                 325       
                                                                 
 activation_2 (Activation)   (None, 5)                 0         
                                                                 
Total params: 62,789
Trainable params: 62,789
Non-trai

In [41]:
# Training model.
checkpoint = ModelCheckpoint('best_model.h5',monitor='val_loss',verbose=True,save_best_only=True)
earlystop = EarlyStopping(monitor='val_acc',patience=10)
hist = model.fit(emb_XTrain,YTrain,batch_size=32,epochs=150,shuffle=True,validation_split=0.1)

Epoch 1/150
4/4 [==============================] - 4s 292ms/step - loss: 1.6064 - accuracy: 0.1780 - val_loss: 1.6236 - val_accuracy: 0.0714
Epoch 2/150
4/4 [==============================] - 0s 23ms/step - loss: 1.5656 - accuracy: 0.3136 - val_loss: 1.6561 - val_accuracy: 0.0714
Epoch 3/150
4/4 [==============================] - 0s 20ms/step - loss: 1.5241 - accuracy: 0.3475 - val_loss: 1.6510 - val_accuracy: 0.1429
Epoch 4/150
4/4 [==============================] - 0s 18ms/step - loss: 1.5054 - accuracy: 0.3559 - val_loss: 1.6579 - val_accuracy: 0.1429
Epoch 5/150
4/4 [==============================] - 0s 18ms/step - loss: 1.4724 - accuracy: 0.3814 - val_loss: 1.6387 - val_accuracy: 0.2143
Epoch 6/150
4/4 [==============================] - 0s 20ms/step - loss: 1.4564 - accuracy: 0.3814 - val_loss: 1.6028 - val_accuracy: 0.2857
Epoch 7/150
4/4 [==============================] - 0s 17ms/step - loss: 1.3801 - accuracy: 0.5000 - val_loss: 1.5794 - val_accuracy: 0.2857
Epoch 8/150
4/4 [==

4/4 [==============================] - 0s 18ms/step - loss: 0.0295 - accuracy: 1.0000 - val_loss: 2.1437 - val_accuracy: 0.6429
Epoch 60/150
4/4 [==============================] - 0s 18ms/step - loss: 0.0188 - accuracy: 1.0000 - val_loss: 2.1690 - val_accuracy: 0.6429
Epoch 61/150
4/4 [==============================] - 0s 18ms/step - loss: 0.0252 - accuracy: 0.9915 - val_loss: 1.6229 - val_accuracy: 0.6429
Epoch 62/150
4/4 [==============================] - 0s 19ms/step - loss: 0.0258 - accuracy: 0.9915 - val_loss: 1.4339 - val_accuracy: 0.6429
Epoch 63/150
4/4 [==============================] - 0s 19ms/step - loss: 0.0205 - accuracy: 1.0000 - val_loss: 1.7457 - val_accuracy: 0.6429
Epoch 64/150
4/4 [==============================] - 0s 18ms/step - loss: 0.0209 - accuracy: 0.9915 - val_loss: 2.0931 - val_accuracy: 0.6429
Epoch 65/150
4/4 [==============================] - 0s 17ms/step - loss: 0.0219 - accuracy: 1.0000 - val_loss: 2.7414 - val_accuracy: 0.5714
Epoch 66/150
4/4 [========

4/4 [==============================] - 0s 18ms/step - loss: 0.0058 - accuracy: 1.0000 - val_loss: 1.9850 - val_accuracy: 0.6429
Epoch 118/150
4/4 [==============================] - 0s 18ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 2.0318 - val_accuracy: 0.6429
Epoch 119/150
4/4 [==============================] - 0s 19ms/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 2.0596 - val_accuracy: 0.6429
Epoch 120/150
4/4 [==============================] - 0s 17ms/step - loss: 0.0051 - accuracy: 1.0000 - val_loss: 2.0868 - val_accuracy: 0.6429
Epoch 121/150
4/4 [==============================] - 0s 18ms/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 2.1120 - val_accuracy: 0.6429
Epoch 122/150
4/4 [==============================] - 0s 17ms/step - loss: 0.0028 - accuracy: 1.0000 - val_loss: 2.1306 - val_accuracy: 0.6429
Epoch 123/150
4/4 [==============================] - 0s 18ms/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 2.1488 - val_accuracy: 0.6429
Epoch 124/150
4/4 [=

In [42]:
# Finding accuracy for our model on testing dataset.
model.evaluate(emb_XTest,YTest)

2/2 [==============================] - 0s 5ms/step - loss: 2.5182 - accuracy: 0.6607


[2.518242597579956, 0.6607142686843872]

P.S. : Reason for an average accuracy on validation data is due to lack of training dataset.